In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, exposure, img_as_float

def get_dark_channel(image, window_size):
    """Computes the dark channel of an image."""
    # Convert the image to grayscale
    gray = np.min(image, axis=2)
    # Compute the minimum value in a local window for each pixel
    dark_channel = np.zeros_like(gray)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            top = max(0, i - window_size // 2)
            left = max(0, j - window_size // 2)
            bottom = min(image.shape[0], i + window_size // 2)
            right = min(image.shape[1], j + window_size // 2)
            dark_channel[i, j] = np.min(gray[top:bottom, left:right])
    return dark_channel

def get_atmosphere(image, dark_channel):
    """Estimates the atmosphere light of an image."""
    # Convert the dark channel to a vector
    dark_channel_vec = dark_channel.ravel()
    # Find the brightest pixels in the dark channel
    indices = dark_channel_vec.argsort()[-int(len(dark_channel_vec) * 0.001):]
    # Compute the average color of these pixels as the atmosphere light
    atmosphere = np.zeros(3)
    for i in indices:
        atmosphere += image.reshape(-1, 3)[i]
    atmosphere /= len(indices)
    return atmosphere

def dehaze(image, atmosphere, t_min=0.1, window_size=15):
    """Removes atmospheric haze from an image using the dark channel prior method."""
    # Convert the image to grayscale
    gray = np.min(image, axis=2)
    # Compute the transmission map using the dark channel prior
    transmission = 1 - 0.95 * get_dark_channel(image / atmosphere, window_size)
    transmission = np.maximum(transmission, t_min)
    # Normalize the transmission map
    transmission_norm = (transmission - np.min(transmission)) / (np.max(transmission) - np.min(transmission))
    # Apply the atmospheric scattering model to the image
    dehazed = np.zeros_like(image)
    for i in range(3):
        dehazed[:,:,i] = (image[:,:,i] - atmosphere[i]) / transmission_norm + atmosphere[i]
    # Clip the dehazed image to [0, 1]
    dehazed = np.clip(dehazed, 0, 1)
    return dehazed

# Load the image
image_path = "S2_India_Jan2022.tif"
image = io.imread(image_path)
# Rescale the image to [0, 1]
image = img_as_float(image)
# Keep only the first 3 channels (RGB) and discard the fourth channel (alpha)
image = image[:, :, :3]
# Compute the dark channel of the image
dark_channel = get_dark_channel(image, window_size=15)
# Estimate the atmosphere light of the image
atmosphere = get_atmosphere(image, dark_channel)
# Dehaze the image using the dark channel prior
dehazed = dehaze(image, atmosphere, t_min=0.1, window_size=15)

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('S5P_NO2_India_2019.png')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Compute the dark channel prior
patch_size = 15
dark_channel = np.zeros_like(gray)
for i in range(gray.shape[0]):
    for j in range(gray.shape[1]):
        patch = gray[max(i-patch_size//2,0):min(i+patch_size//2,gray.shape[0]),
                     max(j-patch_size//2,0):min(j+patch_size//2,gray.shape[1])]
        dark_channel[i,j] = np.min(patch)

# Estimate the atmospheric light
atmosphere = np.percentile(dark_channel, 99)

# Compute the transmission
transmission = 1 - 0.95*dark_channel/atmosphere

# Apply the soft matting algorithm
epsilon = 0.0001
window_size = 15
mean_filter = cv2.blur(transmission, (window_size, window_size))
mean_sqr_filter = cv2.blur(transmission**2, (window_size, window_size))
variance = mean_sqr_filter - mean_filter**2
a = variance / (variance + epsilon)
a = cv2.blur(a, (window_size, window_size))
transmission_matted = a*transmission + (1-a)*mean_filter

# Enhance the image
img_enhanced = np.zeros_like(img)
for c in range(img.shape[2]):
    img_enhanced[:,:,c] = (img[:,:,c] - atmosphere) / np.maximum(transmission_matted, 0.1) + atmosphere

# Save the enhanced image
cv2.imwrite('enhanced_image.png', img_enhanced)


In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('S5P_NO2_India_2019.png')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Compute the dark channel prior
patch_size = 15
dark_channel = np.zeros_like(gray)
for i in range(gray.shape[0]):
    for j in range(gray.shape[1]):
        patch = gray[max(i-patch_size//2,0):min(i+patch_size//2,gray.shape[0]),
                     max(j-patch_size//2,0):min(j+patch_size//2,gray.shape[1])]
        dark_channel[i,j] = np.min(patch)

# Estimate the atmospheric light
atmosphere = np.percentile(dark_channel, 99)

# Compute the transmission
transmission = 1 - 0.95*dark_channel/atmosphere

# Apply the soft matting algorithm
epsilon = 0.0001
window_size = 15
mean_filter = cv2.blur(transmission, (window_size, window_size))
mean_sqr_filter = cv2.blur(transmission**2, (window_size, window_size))
variance = mean_sqr_filter - mean_filter**2
a = variance / (variance + epsilon)
a = cv2.blur(a, (window_size, window_size))
transmission_matted = a*transmission + (1-a)*mean_filter

# High-pass filter the image using the transmission map
alpha = 0.1
img_filtered = img - cv2.GaussianBlur(img, (0, 0), np.mean(transmission_matted)*alpha)

# Add the filtered image to the original image
img_sharp = cv2.add(img, img_filtered)

# Save the sharpened image
cv2.imwrite('sharpened_image.png', img_sharp)
